## route()

**route()** es un decorador que conecta una función a una URL, en este caso conectamos a la ruta 
/hola a la función hello().Esto es llamado una ruta, y es uno de los conceptos mas importantes 
de este framework. Se pueden definir tantas rutas como sean necesarias, y cuando desde un
navegador solicite una URL, la funcion asociada a esa ruta será ejecutada y responderá al 
navegador.

La función **run()** en las ultimas lineas inicia el servidor web de 
desarrollo, este se ejecuta en local en el puerto 8080(puede definirse
el puerto), y responde a las solicitudes hasta que presionamos ctrl-C.
Podemos cambiar el servidor , esto en caso de llevar a "producción"
nuestra aplicación, pero para desarrollo con las opciones default nos
es suficiente, no necesita instalarse y es lo mas sencillo para probar
nuestra aplicación.

In [ ]:
pip install bottle

In [ ]:
from bottle import route, run

@route('/hola')
def hello():
        return "Hola Mundo!"
run(host='0.0.0.0', port=5000, debug=True)

## La aplicación default

Podemos observar en el ejemplo que usamos el decorador **route()** para
definir las rutas , esto añade las rutas a nuestra aplicacion default,
una instancia que se crea la primera vez que usamos el decorador 
**route()**, pero para seguir un enfoque mas orientado a objetos,
crearemos primero la instancia de la aplicación y añadiremos estas
rutas a nuestra instancia explicitamente.

In [ ]:
from bottle import Bottle, run

app = Bottle()

@app.route('/hola')
def hello():
        return "Hola Mundo!"
run(app, host='0.0.0.0', port=5000, debug=True)

## Añadiendo una ruta mas

Podemos añadir mas de una ruta a una función, además podemos añadir
**"comodines"(wildcards)** a la ruta para hacerla mas dinámica. Además
veremos en acción el modulo template, donde nuestra función regresará
un valor y será reemplazado en nuestro HTML.

In [ ]:
from bottle import Bottle, run, template

app = Bottle()

@app.route('/')
@app.route('/hola/<name>')
def hello(name='pinche putita'):
        return template('Hola {{name}}, cómo estás?', name=name)
run(app, host='0.0.0.0', port=5000, debug=True)

In [ ]:
from bottle import Bottle, run, template, request, redirect

app = Bottle()

@app.route('/')
@app.route('/hola/<name>')
def hello(name='Extraño'):
        return template('Hola {{name}}, cómo estás?', name=name)

@app.route('/logged/<name>')
def logged(name='Extraño'):
    return template('''you are in! {{name}}''',name=name)

def checklogin(username, password):
    isLogged=False
    if username == "Agustin" and password == '123':
        isLogged=True
    return isLogged

@app.route('/login', method='GET')
def login():
    isFailed= '0' if request.query.Failed == '' else request.query.Failed
    
    formString= '''<form action="/login" method="post">
        Username: <input name="username" type="text" />
        Password: <input name="password" type="password" />
        <input value="login" type="submit" />
        </form>
    '''
    if int(isFailed) == 1:
        return '<p><strong>Login failed! </p>'+formString
    else:
        return formString

@app.post('/login') # or @route('/login', method='POST')
def do_login():
    username = request.forms.get('username')
    password = request.forms.get('password')
    if checklogin(username, password):
        redirect(f"/logged/{username}")
    else:
        redirect(f"/login?Failed=1")     
    
run(app, host='0.0.0.0', port=5000, debug=True)


Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:5000/
Hit Ctrl-C to quit.

10.0.2.2 - - [15/Mar/2019 11:05:26] "GET / HTTP/1.1" 200 28
10.0.2.2 - - [15/Mar/2019 11:05:32] "GET /login HTTP/1.1" 200 219
10.0.2.2 - - [15/Mar/2019 11:05:40] "POST /login HTTP/1.1" 303 0
10.0.2.2 - - [15/Mar/2019 11:05:40] "GET /logged/Agustin HTTP/1.1" 200 19
